# Load Libraries
Load libraries and initialize Prompt Registry Client

In [4]:
# Import necessary classes
from gen_ai_hub.proxy import get_proxy_client
from gen_ai_hub.prompt_registry.client import PromptTemplateClient
from dotenv import load_dotenv
load_dotenv()

# Initialize the proxy client which handles authentication based on env vars/config file
proxy_client = get_proxy_client(proxy_version="gen-ai-hub")

# Initialize the PromptTemplateClient, passing the proxy client and AI Core base URL
# The base URL is typically retrieved from the proxy client's AI Core client instance
prompt_registry_client = PromptTemplateClient(proxy_client=proxy_client)

print("PromptTemplateClient initialized successfully.")

PromptTemplateClient initialized successfully.


## Creating a Prompt Template
Creating a template involves defining its structure using `PromptTemplateSpec` and `PromptTemplate` objects and then calling the `create_prompt_template` method. The `PromptTemplate` object defines the role (system, user, or assistant) and the content for a part of the prompt.

In [ ]:
# Import models for defining the template structure
from gen_ai_hub.prompt_registry.models.prompt_template import PromptTemplateSpec, PromptTemplate

# Define the template specification: a list of PromptTemplate objects
# This example creates a simple system prompt
prompt_template_spec = PromptTemplateSpec(template=[PromptTemplate(role='system',
                                                                   content='You are a helpful assistant.')])


# Call the create method with scenario, name, version, and the spec
# Scenario, name, and version act as identifiers for the template group
response = prompt_registry_client.create_prompt_template(
    scenario='MyScenario',          # A logical grouping for your templates
    name='my_first_template',     # A unique name within the scenario/version
    version='1.0.0',              # A version identifier (e.g., semantic version)
    prompt_template_spec=prompt_template_spec
)

# The response contains the unique ID of the created template version
template_id = response.id
print(f"Created Prompt Template Version with ID: {template_id}")
print(f"Response Message: {response.message}") 

Created Prompt Template Version with ID: e64aac9c-72f8-4b27-828a-31c5e1592e18
Response Message: Prompt updated successfully.


# Retrieving a Prompt Template
To fetch the details of a specific template version, use the `get_prompt_template_by_id` method, providing the unique `template_id` obtained during creation or modification. The response object contains the full details of that specific template version, including its specification (spec), identifiers, and metadata

In [6]:
# Retrieve the template details using its unique version ID
response = prompt_registry_client.get_prompt_template_by_id(template_id)

# Access the template specification (structure and content) from the response
if response and response.spec:
    print("Retrieved Template Spec:", response.spec.template)
else:
    print(f"Could not retrieve template with ID: {template_id}")

Retrieved Template Spec: [PromptTemplate(role='system', content='You are a helpful assistant.')]


# Modifying a Prompt Template
Modifying an existing prompt template in the Prompt Registry using the SDK typically involves creating a new version rather than altering the existing one in place. This is achieved by calling `create_prompt_template` again, using the same `scenario`, `name`, and `version` identifiers, but providing an updated `prompt_template_spec`. This design promotes immutability and ensures a complete history of changes.

In [7]:
# Import the necessary models again (if in a new context)
from gen_ai_hub.prompt_registry.models.prompt_template import PromptTemplateSpec, PromptTemplate

# Define the updated spec with a variable placeholder
prompt_template_spec_v2 = PromptTemplateSpec(template=[PromptTemplate(role='system',
                                                                      content='You are a helpful assistant for {{ ?topic }}.')])

# Call create_prompt_template again with the SAME identifiers but the NEW spec
response_v2 = prompt_registry_client.create_prompt_template(
    scenario='MyScenario',
    name='my_first_template',
    version='1.0.0', # Version identifier remains the same for history grouping
    prompt_template_spec=prompt_template_spec_v2
)

# This call returns a NEW unique ID for the updated version
input_template_id = response_v2.id
print(f"Created Updated Prompt Template Version with ID: {input_template_id}")
print("Update Message:", response_v2.message) # Confirmation message

Created Updated Prompt Template Version with ID: 8735721b-4952-4e29-a37d-9bff67041eea
Update Message: Prompt updated successfully.


It is important to note that `input_template_id` now refers to the latest version containing the variable, while the original `template_id` still points to the first version. The version parameter (`1.0.0` in this case) acts as a label to group these related historical versions together, rather than strictly enforcing semantic versioning rules within the API call itself.

# Viewing Template History
To retrieve the history of all versions created for a specific template (grouped by `scenario`, `name`, and `version`), use the `get_prompt_template_history` method.

In [11]:
# Retrieve the history using the common identifiers
history_response = prompt_registry_client.get_prompt_template_history(
    scenario='MyScenario',
    name='my_first_template',
    version='1.0.0'
)

# The response object contains the history;.json() provides a serializable view
# The exact structure should be inspected based on the SDK version
# It typically contains a list of versions with their IDs and creation timestamps.
print("Template History (JSON representation):")
print(history_response.model_dump_json())

Template History (JSON representation):
{"count":4,"resources":[{"id":"8735721b-4952-4e29-a37d-9bff67041eea","name":"my_first_template","version":"1.0.0","scenario":"MyScenario","creation_timestamp":"2025-04-29T12:22:02.720000","managed_by":"imperative","is_version_head":true,"spec":{"template":[{"role":"system","content":"You are a helpful assistant for {{ ?topic }}."}],"defaults":{},"additional_fields":{}}},{"id":"e64aac9c-72f8-4b27-828a-31c5e1592e18","name":"my_first_template","version":"1.0.0","scenario":"MyScenario","creation_timestamp":"2025-04-29T12:18:30.758000","managed_by":"imperative","is_version_head":false,"spec":{"template":[{"role":"system","content":"You are a helpful assistant."}],"defaults":{},"additional_fields":{}}},{"id":"d9b51e4e-3314-4a95-b525-70bd726d0958","name":"my_first_template","version":"1.0.0","scenario":"MyScenario","creation_timestamp":"2025-04-29T12:03:45.413000","managed_by":"imperative","is_version_head":false,"spec":{"template":[{"role":"system","co

# Filling a Prompt Template
Once a template with variables is created (like the one with `input_template_id` above), it can be "filled" with specific values using the `fill_prompt_template_by_id` method. This substitutes the placeholders with the provided data.

In [12]:
# Fill the template version that includes the 'topic' variable
fill_response = prompt_registry_client.fill_prompt_template_by_id(
    template_id=input_template_id, # Use the ID of the specific version to fill
    input_params={"topic": "chemistry"} # Dictionary mapping variable names to values
)

# The response contains the 'parsed_prompt' attribute with the filled content
if fill_response and fill_response.parsed_prompt:
    print("Filled Prompt:", fill_response.parsed_prompt)
    # This filled prompt string can now be sent to an LLM via the appropriate SDK client
else:
    print(f"Could not fill template with ID: {input_template_id}")

Filled Prompt: [PromptTemplate(role='system', content='You are a helpful assistant for chemistry.')]
